# Wikipediaから一覧表を取得
- 実は Excel から直接URL開いて表をimportすれば一瞬で済む話だけれど…。

In [ ]:
__author__ = "Admin GGCS"
__copyright__ = "Copyright 2019"
__license__ = "MIT"
__version__ = "1.0.0"
__maintainer__ = "Admin GGCS"
__website__ = "ggcs.io"

#Setup
import requests
import bs4
#import os
#import time
#import webbrowser
#import shutil
#import pandas as pd

#DOWNLOAD_DIR = os.path.join(".", "DownloadedHtml")
#if not os.path.isdir(DOWNLOAD_DIR):
#    os.makedirs(DOWNLOAD_DIR)

## 目的のURLを入力

In [ ]:
#Input a target URL
try:
    TARGET_URL = input('Target URL? >> ')
except Exception as exc_msg:
    print('Error!: {}'.format(exc_msg))

#目的のURLを聞かれるので、https://ja.wikipedia.org/wiki/foo みたいな感じで入力。

## HTMLの様子を把握、どうやって抽出するか見当を付ける

In [ ]:
#Get the page data
r = requests.get(TARGET_URL)
#r.encoding = 'foo' #Garble killer. もし文字化けしたらfooを適当に変えてみる。
soup = bs4.BeautifulSoup(r.text, "lxml")
print(soup.prettify())

## td 要素抜き出し

In [ ]:
tdList = soup.select('td')

#十中八九、途中でセルのマージ等があるので一気通貫では処理できない。Blockに分けて試行錯誤しながら適当に設定。
#これらの数字はあくまで一例。
firstBlockStarts = 2
secondBlockStarts = 371
thirdBlockStarts = 480
blockEnds = 1301

for i in range(firstBlockStarts, secondBlockStarts, 10): #10要素ずつスキップ
    print(tdList[i].text, tdList[i+1].text, tdList[i+2].text) #頭から i, i+1, 1+2 の 3列だけ取得
for i in range(secondBlockStarts, thirdBlockStarts, 10):
    print(tdList[i].text, tdList[i+1].text, tdList[i+2].text)
for i in range(thirdBlockStarts, blockEnds, 10):
    print(tdList[i].text, tdList[i+1].text, tdList[i+2].text)

## データをファイルに書き出したい場合
- 上の Cell を実行して Start / End を設定してあることが前提

### CSVに書き出したい場合
- UTF-8, BOM無しなので、Excel で開く場合は開くときにExcelの encoding を手動で UTF-8 に設定してください。

In [ ]:
import os

#Current working directory 直下にあらかじめ出力ファイル用のフォルダを作っておく
OUTPUT_DIR = os.path.join(".", "outputs")
if not os.path.isdir(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

#抽出結果を string 形式で用意（コンマ区切り、改行でレコードを分ける）
f = 'Column A, Column B, Column C, \n'
for i in range(firstBlockStarts, secondBlockStarts, 10): 
    f += ','.join((tdList[i].text, tdList[i+1].text, tdList[i+2].text, '\n'))
for i in range(secondBlockStarts, thirdBlockStarts, 10):
    f += ','.join((tdList[i].text, tdList[i+1].text, tdList[i+2].text, '\n'))
for i in range(thirdBlockStarts, blockEnds, 10):
    f += ','.join((tdList[i].text, tdList[i+1].text, tdList[i+2].text, '\n'))

#output.csv を作って str(f) を書き込み、保存
with open(os.path.join(OUTPUT_DIR, 'output.csv'),'w') as outputWrapper:
    outputWrapper.write(str(f))
    
#アウトプットフォルダを開く
os.system('open ' + OUTPUT_DIR)
print('Done. You can find output.csv in ', OUTPUT_DIR)   

### Excelに書き出したい場合

In [ ]:
import os
import pandas as pd

#Current working directory 直下にあらかじめ出力ファイル用のフォルダを作っておく
OUTPUT_DIR = os.path.join(".", "outputs")
if not os.path.isdir(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

f = []
for i in range(firstBlockStarts, secondBlockStarts, 10): 
    f += [[tdList[i].text, tdList[i+1].text, tdList[i+2].text]]
for i in range(secondBlockStarts, thirdBlockStarts, 10):
    f += [[tdList[i].text, tdList[i+1].text, tdList[i+2].text]]
for i in range(thirdBlockStarts, blockEnds, 10):
    f += [[tdList[i].text, tdList[i+1].text, tdList[i+2].text]]

#Pandas Data Frame を経由して Excel に保存
df = pd.DataFrame(f, columns=['Column A', 'Column B', 'Column C'])
df.to_excel(os.path.join(OUTPUT_DIR, 'output.xlsx'), index=False)

#アウトプットフォルダを開く
os.system('open ' + OUTPUT_DIR)
print('Done. You can find output.xlsx in ', OUTPUT_DIR)   